<a href="https://colab.research.google.com/github/BenyaminZojaji/Deep_Learning/blob/main/PyTorch%20PersianMnist/PersianMnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from tqdm import tqdm

In [ ]:
def calc_acc(preds, labels):
  _, preds_max = torch.max(preds, 1)
  acc = torch.sum(preds_max == labels.data, dtype=torch.float64) / len(preds)
  return acc

In [ ]:
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    # cuz they are RGB we put 3
    self.conv1 = nn.Conv2d(3, 32, (3, 3), (1, 1), (1, 1))
    self.conv2 = nn.Conv2d(32, 32, (3, 3), (1, 1), (1, 1))
    self.conv3 = nn.Conv2d(32, 64, (3, 3), (1, 1), (1, 1))
    self.conv4 = nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1))

    self.fc1 = nn.Linear(64*8*8, 512) # batch_size * size * size (size after convs and maxpoolings)
    self.fc2 = nn.Linear(512, 10)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.max_pool2d(x, kernel_size=(2, 2))
    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x, kernel_size=(2, 2))
    x = F.relu(self.conv3(x))
    x = F.max_pool2d(x, kernel_size=(2, 2))
    x = F.relu(self.conv4(x))
    # print(x.shape)
    x = torch.flatten(x, start_dim=1) # cuz we dont want to flatten all of it! (0 is batch size)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    x = torch.softmax(x, dim=1)
    return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model().to(device)

In [ ]:
# Hyper Parameters
batch_size = 64
epochs = 10
lr = 0.001

In [ ]:
transform = torchvision.transforms.Compose([
                                            transforms.ToPILImage(), # added because of the Error: img should be PIL Image. Got <class 'numpy.ndarray'>
                                            transforms.RandomRotation(10),
                                            transforms.Resize((70, 70)),
                                            transforms.ToTensor(),
                                            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

dataset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/Dataset/MNIST_persian', transform=transform)
train_data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# compile
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_function = nn.CrossEntropyLoss()

In [ ]:
model.train()

for epoch in range(epochs):
  train_loss = 0
  train_acc = 0
  for images, labels in tqdm(train_data_loader):
    images, labels = images.to(device), labels.to(device)
    optimizer.zero_grad()

    preds = model(images)

    loss = loss_function(preds, labels)
    loss.backward()
    optimizer.step()

    train_loss += loss
    train_acc += calc_acc(preds, labels)
  
  total_loss = train_loss / len(train_data_loader)
  total_acc = train_acc / len(train_data_loader)

  print(f'Epoch: {epoch}, Loss: {total_loss}, Acc:{total_acc}')

100%|██████████| 19/19 [02:48<00:00,  8.88s/it]


Epoch: 0, Loss: 2.288764238357544, Acc:0.14336622807017543


100%|██████████| 19/19 [00:03<00:00,  5.06it/s]


Epoch: 1, Loss: 2.0252838134765625, Acc:0.4454495614035087


100%|██████████| 19/19 [00:03<00:00,  4.96it/s]


...

Epoch: 7, Loss: 1.7002999782562256, Acc:0.7606907894736842


100%|██████████| 19/19 [00:03<00:00,  4.78it/s]


Epoch: 8, Loss: 1.6874136924743652, Acc:0.7798793859649122


100%|██████████| 19/19 [00:03<00:00,  5.00it/s]

Epoch: 9, Loss: 1.6771321296691895, Acc:0.7834429824561403


In [ ]:
# save
torch.save(model.state_dict(), 'PersianMNIST.pth')

In [ ]:
# inference

import cv2
import numpy as np
import time

#model.train(False)
model.eval()

# preprocess
img = cv2.imread('/content/photo_2022-05-04_14-43-10.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (70, 70))
tensor = transform(img).unsqueeze(0).to(device)
#tensor = transform(img)

# process
start = time.time()
preds = model(tensor)
end = time.time()

# postprocess
# _, output=torch.max(preds, 1)
preds = preds.cpu().detach().numpy()
output = np.argmax(preds)
print(f'{output+1}, Process-Time: {end-start}')

1, Process-Time: 0.002031087875366211
